## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

### Obs
- Apesar de o curso estar estruturado em MySql, daremos sequência aqui com a integração ao **SQL Server**, que é mais familiar e confortável

In [1]:
import pyodbc 

try:
    print('Tentando conectar ao banco Sql Server...')
    conn = pyodbc.connect('DRIVER=SQL Server;Server=DESKTOP-E29RO7M\\SQLEXPRESS;Database=dbprodutos')
    print('Conexão bem-sucedida!')
except Exception as e:
    print('A conexão falhou.')
    print(e)


Tentando conectar ao banco Sql Server...
Conexão bem-sucedida!


Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [3]:
# definindo o objeto cursor (responsável por permitir a manipualação de tabelas e registros no banco de dados através do python)
cursor = conn.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [23]:
for db in cursor.execute('''
                         SELECT name, database_id, create_date  
                            FROM sys.databases
'''):
    print(db.name)

master
tempdb
model
msdb
Lista 1 - Exercícios
SQL_DB_1
Universidade-Exercício
AdventureWorks
ProjetosFrontEnd
CursoSqlAlchemy
dbprodutos


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [24]:
import pandas as pd 

df_livros = pd.read_csv('./data/tabela_livros.csv')
display(df_livros)

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,667f3fd483e95f42cebda945,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,667f3fd483e95f42cebda946,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,667f3fd483e95f42cebda958,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,667f3fd483e95f42cebda966,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,667f3fd483e95f42cebda968,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,667f3fd483e95f42cebdcde0,Dashboards com Power BI,livros,41.27,2.728608,2022-07-31,João Souza,SC,5,cartao_credito,12,-27.45,-50.95
738,667f3fd483e95f42cebdcde4,Ciência de dados com python,livros,81.58,4.632666,2020-08-31,Juliana Costa,SP,5,cartao_credito,2,-22.19,-48.79
739,667f3fd483e95f42cebdcdea,Modelagem preditiva,livros,109.17,6.701389,2021-08-31,Felipe Santos,MG,3,cartao_credito,3,-18.10,-44.38
740,667f3fd483e95f42cebdcdf6,Modelagem preditiva,livros,94.89,5.578872,2022-08-31,Pedro Gomes,SP,5,cartao_credito,1,-22.19,-48.79


In [25]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [26]:
df_livros.shape

(742, 13)

In [27]:
cursor.execute('''
    create table TabelaLivros 
    (
        Id varchar(1000) primary key,
        Produto varchar(1000),
        CategoriaProduto varchar(1000),
        Preco decimal(18,2),
        Frete decimal(18,6),
        DataCompra date,
        Vendedor varchar(1000),
        LocalCompra varchar(1000),
        AvaliacaoCompra int,
        TipoPagamento varchar(1000),
        QtdParcelas int,
        Latitude decimal(18,2),
        Longitude decimal(18,2)
    )
''')
cursor.commit()

**Selecionando a base de dados para verificar a tabela criada**

In [28]:
cursor.execute("select distinct Name from sys.Tables")
for item in cursor:
    print(item)

('TabelaLivros',)
('TabelaProdutos',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [29]:
for i, row in df_livros.iterrows():
    if i in [0,1,2,3,4]:
        print(tuple(row))

('667f3fd483e95f42cebda945', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('667f3fd483e95f42cebda946', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('667f3fd483e95f42cebda958', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('667f3fd483e95f42cebda966', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('667f3fd483e95f42cebda968', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)


In [30]:
lista_registros = [tuple(row) for i, row in df_livros.iterrows()]

In [31]:
sql_query = '''
    insert into TabelaLivros (
        Id
        , Produto
        , CategoriaProduto
        , Preco
        , Frete
        , DataCompra
        , Vendedor
        , LocalCompra
        , AvaliacaoCompra
        , TipoPagamento
        , QtdParcelas
        , Latitude
        , Longitude
        ) 
    values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
# os ? representam parâmetros dinâmicos no contexto dessa conexão (via PyOdbc)

cursor.executemany(sql_query, lista_registros)
cursor.commit()

In [32]:
a = cursor.execute('select count(*) from TabelaLivros').fetchall() # o fetchall converte o objeto do tipo cursor em uma lista de tuplas (cada linha da consulta se torna uma tupla)
print(a)

[(742,)]


- Tratamento e insert de dados do arquivo de produtos

In [33]:
# gerando a tabela sql que será responsável por receber esses registros
#_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude

query_sql = '''
    drop table if exists dbo.TabelaProdutos;
    
    create table dbo.TabelaProdutos 
    (
        Id varchar(1000) not null primary key,
        Produto varchar(1000),
        CategoriaProduto varchar(1000),
        Preco decimal(18,2),
        Frete decimal(18,2),
        DataCompra date,
        Vendedor varchar(1000),
        LocalCompra varchar(1000),
        AvaliacaoCompra int,
        TipoPagamento varchar(1000),
        QuantidadeParcelas int,
        Latitude float,
        Longitude float
    )
'''

cursor.execute(query_sql)
conn.commit()

In [34]:
import pandas as pd 

df_produtos = pd.read_csv('data/tabela_produtos.csv', sep= ',')
df_produtos.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,667f3fd483e95f42cebda94e,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,667f3fd483e95f42cebda94f,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,667f3fd483e95f42cebda950,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,667f3fd483e95f42cebda951,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,667f3fd483e95f42cebda952,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [35]:
for i, row in df_produtos.iterrows():
    if i <= 5:
        print(tuple(row))

('667f3fd483e95f42cebda94e', 'Xadrez de madeira', 'brinquedos', 25.23, 0.0, '2021-01-01', 'Thiago Silva', 'BA', 5, 'cartao_credito', 2, -13.29, -41.71)
('667f3fd483e95f42cebda94f', 'Impressora', 'eletronicos', 322.04, 14.7321002401, '2021-01-01', 'João Souza', 'SP', 3, 'cartao_credito', 1, -22.19, -48.79)
('667f3fd483e95f42cebda950', 'Mesa de centro', 'moveis', 282.22, 12.6118050347, '2021-01-01', 'João Souza', 'SP', 3, 'boleto', 1, -22.19, -48.79)
('667f3fd483e95f42cebda951', 'Tablet ABXY', 'eletronicos', 1100.42, 68.4513476183, '2021-01-01', 'Beatriz Moraes', 'SP', 1, 'cupom', 1, -22.19, -48.79)
('667f3fd483e95f42cebda952', 'Fogão', 'eletrodomesticos', 791.81, 42.4446263246, '2021-01-01', 'Juliana Costa', 'SP', 4, 'boleto', 1, -22.19, -48.79)
('667f3fd483e95f42cebda953', 'Violão', 'instrumentos musicais', 316.84, 17.1595536296, '2022-01-01', 'Pedro Gomes', 'SP', 5, 'cartao_credito', 2, -22.19, -48.79)


In [36]:
lista_produtos = [tuple(row) for i, row in df_produtos.iterrows()]
print(lista_produtos[:5])

[('667f3fd483e95f42cebda94e', 'Xadrez de madeira', 'brinquedos', 25.23, 0.0, '2021-01-01', 'Thiago Silva', 'BA', 5, 'cartao_credito', 2, -13.29, -41.71), ('667f3fd483e95f42cebda94f', 'Impressora', 'eletronicos', 322.04, 14.7321002401, '2021-01-01', 'João Souza', 'SP', 3, 'cartao_credito', 1, -22.19, -48.79), ('667f3fd483e95f42cebda950', 'Mesa de centro', 'moveis', 282.22, 12.6118050347, '2021-01-01', 'João Souza', 'SP', 3, 'boleto', 1, -22.19, -48.79), ('667f3fd483e95f42cebda951', 'Tablet ABXY', 'eletronicos', 1100.42, 68.4513476183, '2021-01-01', 'Beatriz Moraes', 'SP', 1, 'cupom', 1, -22.19, -48.79), ('667f3fd483e95f42cebda952', 'Fogão', 'eletrodomesticos', 791.81, 42.4446263246, '2021-01-01', 'Juliana Costa', 'SP', 4, 'boleto', 1, -22.19, -48.79)]


In [37]:
query_sql = '''
    insert into TabelaProdutos (
        Id,
        Produto,
        CategoriaProduto,
        Preco,
        Frete,
        DataCompra,
        Vendedor,
        LocalCompra,
        AvaliacaoCompra,
        TipoPagamento,
        QuantidadeParcelas,
        Latitude,
        Longitude
    )
    values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

cursor.executemany(query_sql, lista_produtos)
conn.commit()

In [38]:
cursor.execute('select * from dbo.TabelaProdutos')


In [43]:
df = pd.read_sql('select * from dbo.TabelaProdutos', conn)
df.set_index('Id', inplace=True)

df.head()

C:\Users\LUCAS\AppData\Local\Temp\ipykernel_20112\1455657817.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from dbo.TabelaProdutos', conn)


,Produto,CategoriaProduto,Preco,Frete,DataCompra,Vendedor,LocalCompra,AvaliacaoCompra,TipoPagamento,QuantidadeParcelas,Latitude,Longitude
Id,,,,,,,,,,,,
667f3fd483e95f42cebda94e,Xadrez de madeira,brinquedos,25.23,0.00,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
667f3fd483e95f42cebda94f,Impressora,eletronicos,322.04,14.73,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
667f3fd483e95f42cebda950,Mesa de centro,moveis,282.22,12.61,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
667f3fd483e95f42cebda951,Tablet ABXY,eletronicos,1100.42,68.45,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
667f3fd483e95f42cebda952,Fogão,eletrodomesticos,791.81,42.44,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


## Visualizando os dados inseridos

In [44]:
cursor.execute('select * from TabelaLivros')

for i in cursor:
    print(i)

('667f3fd483e95f42cebda945', 'Modelagem preditiva', 'livros', Decimal('92.45'), Decimal('5.609697'), '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, Decimal('-13.29'), Decimal('-41.71'))
('667f3fd483e95f42cebda946', 'Iniciando em programação', 'livros', Decimal('43.84'), Decimal('0.000000'), '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, Decimal('-22.19'), Decimal('-48.79'))
('667f3fd483e95f42cebda958', 'Iniciando em programação', 'livros', Decimal('63.25'), Decimal('3.894137'), '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, Decimal('-22.25'), Decimal('-42.66'))
('667f3fd483e95f42cebda966', 'Ciência de dados com python', 'livros', Decimal('86.13'), Decimal('5.273176'), '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, Decimal('-22.25'), Decimal('-42.66'))
('667f3fd483e95f42cebda968', 'Ciência de dados com python', 'livros', Decimal('72.75'), Decimal('1.458158'), '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, Decimal(

In [46]:
linhas_query = cursor.execute('select * from TabelaLivros').fetchall()

for linha in linhas_query[:5]:
    print(linha)

('667f3fd483e95f42cebda945', 'Modelagem preditiva', 'livros', Decimal('92.45'), Decimal('5.609697'), '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, Decimal('-13.29'), Decimal('-41.71'))
('667f3fd483e95f42cebda946', 'Iniciando em programação', 'livros', Decimal('43.84'), Decimal('0.000000'), '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, Decimal('-22.19'), Decimal('-48.79'))
('667f3fd483e95f42cebda958', 'Iniciando em programação', 'livros', Decimal('63.25'), Decimal('3.894137'), '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, Decimal('-22.25'), Decimal('-42.66'))
('667f3fd483e95f42cebda966', 'Ciência de dados com python', 'livros', Decimal('86.13'), Decimal('5.273176'), '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, Decimal('-22.25'), Decimal('-42.66'))
('667f3fd483e95f42cebda968', 'Ciência de dados com python', 'livros', Decimal('72.75'), Decimal('1.458158'), '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, Decimal(

In [53]:
columns_names = ['Id', 'Produto', 'CategoriaProduto', 'Preco', 'Frete', 'DataCompra', 'Vendedor', 'LocalCompra', 'AvaliacaoCompra', 'TipoPagamento', 'QuantidadeParcelas', 'Latitude', 'Longitude']

df_data = []

for linha in linhas_query:
    aux_dict = {}
    for i in range(len(columns_names)):
        aux_dict[columns_names[i]] = linha[i]
    
    df_data.append(aux_dict)

df = pd.DataFrame(data= df_data)
df.head()

,Id,Produto,CategoriaProduto,Preco,Frete,DataCompra,Vendedor,LocalCompra,AvaliacaoCompra,TipoPagamento,QuantidadeParcelas,Latitude,Longitude
0,667f3fd483e95f42cebda945,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,667f3fd483e95f42cebda946,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,667f3fd483e95f42cebda958,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,667f3fd483e95f42cebda966,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,667f3fd483e95f42cebda968,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55
